In [1]:
import pandas as pd
import numpy as np
import re
import datetime


DTR = input("Input consolidated DTR")
DTR = pd.read_excel(DTR)

#-------- cash deposit display ------------------------------------------------------------------------------------------------------
cash = DTR.copy()
cash1 = cash[cash["txn_code"] == 300]
cash12 = cash[cash["txn_code"] == 640]
frames = [cash1, cash12]
cash_late = pd.concat(frames)
cash_late["tr_time"]
cash_late["tr_time"] = cash_late["tr_time"].astype('datetime64[ns]')

cash_late['time']= cash_late["tr_time"].dt.time
time_Cash = cash_late.set_index('tr_time').between_time('18:30:00', '23:00:00')
deposit_late = time_Cash[['unit_id', 'tnx_date', 'tnx_no', 'batch_no', 'acc_unit_id',
       'account_no', 'account_title', 'post_date', 'value_date',
       'release_date', 'txn_code', 'currency_code', 'amount', 'local_eqv',
       'ex_rate', 'ex_rate_code', 'vch_narration', 'charge_code',
       'trans_amt_type', 'drcr_flag', 'doc_id', 'instrument_no', 'narrative',
       'trans_type', 'product_code', 'emp_id', 'time']].reset_index()

#---------------- withdraawl delay -----------------------------------------------------------------------------------------------------

cash = DTR.copy()
w1 = cash[cash["txn_code"] == 319]
w2 = cash[cash["txn_code"] == 651]
frames_w = [w1, w2]
w_late = pd.concat(frames_w)
w_late["tr_time"]
w_late["tr_time"] = w_late["tr_time"].astype('datetime64[ns]')

w_late['time']= w_late["tr_time"].dt.time
time_Cash_w = w_late.set_index('tr_time').between_time('18:30:00', '23:00:00')
w_late = time_Cash_w[['unit_id', 'tnx_date', 'tnx_no', 'batch_no', 'acc_unit_id',
       'account_no', 'account_title', 'post_date', 'value_date',
       'release_date', 'txn_code', 'currency_code', 'amount', 'local_eqv',
       'ex_rate', 'ex_rate_code', 'vch_narration', 'charge_code',
       'trans_amt_type', 'drcr_flag', 'doc_id', 'instrument_no', 'narrative',
       'trans_type', 'product_code', 'emp_id', 'time']].reset_index()

#--------- Inward Clearing --------------------------------------------------------------------------------------------------------
inward = DTR.copy()
inw = inward[inward["txn_code"] == 349]

inw["tr_time"] =inw["tr_time"].astype('datetime64[ns]')
inw['time']= inw["tr_time"].dt.time
inw_c =inw.set_index('tr_time').between_time('13:30:00', '23:00:00')

inw_c.reset_index(inplace= True)

inward_clearing = inw_c[inw_c["trans_type"]=='INCLR' ]

#----------- STAN NO 641 -----------------------------------------------------------------------------------------------------------

stan = DTR.copy()
stan["account_no"]=stan["account_no"].astype(str)
stan_gl= stan.query("account_no.str.len() == 8", engine="python")
stangl_41 = stan_gl[stan_gl["txn_code"] == 641]
stan_no_641 = stangl_41.query("vch_narration.str.len() <= 10", engine="python")

#-------- CD CTR without GL -------------------------------------------------------------------------------------------------------

ctr = DTR.copy()
ctr1 = ctr[ctr["txn_code"] == 300]
ctr2 = ctr[ctr["txn_code"] == 640]
framesctr = [ctr1, ctr2]
ctr_con = pd.concat(framesctr)

#--------------------------------------------------------

ctr_con["account_no"]=ctr_con["account_no"].astype(str)
ctr_con= ctr_con.query("account_no.str.len() > 8", engine="python")
#--------------------------------------------------------
ctr_con["transaction_count"] = ctr_con.groupby(["account_no", "tnx_date"])["account_no"].transform("size")
#--------------------------------------------------------
data2 = ctr_con[ctr_con["transaction_count"] != 1]
#--------------------------------------------------------
data2["sum"]= data2.groupby(["account_no", "tnx_date"])["local_eqv"].transform(sum)
#--------------------------------------------------------
twomil = data2[data2["sum"] >= 2000000 ]


#---------------CD CTR WITH GL---------------------------------------------------------------------------------------------------------
ctr = DTR.copy()
ctr1 = ctr[ctr["txn_code"] == 300]
ctr2 = ctr[ctr["txn_code"] == 640]
framesctr = [ctr1, ctr2]
ctr_con = pd.concat(framesctr)

#--------------------------------------------------------

ctr_con["account_no"]=ctr_con["account_no"].astype(str)

#--------------------------------------------------------
ctr_con["transaction_count"] = ctr_con.groupby(["account_no", "tnx_date"])["account_no"].transform("size")
#--------------------------------------------------------
data2 = ctr_con[ctr_con["transaction_count"] != 1]
#--------------------------------------------------------
data2["sum"]= data2.groupby(["account_no", "tnx_date"])["local_eqv"].transform(sum)
#--------------------------------------------------------
data2["sum"] =data2["sum"].abs()
#--------------------------------------------------------
twomilgl = data2[data2["sum"] >= 2000000 ]
twomilgl.shape

#----------------- CW CTR WITHOUT GL ----------------------------------------------------------------------------------------------------------------
ctrcw = DTR.copy()
ctr1cw = ctrcw[ctrcw["txn_code"] == 319]
ctr2cw = ctrcw[ctrcw["txn_code"] == 651]
framesctr = [ctr1cw, ctr2cw]
ctr_concw = pd.concat(framesctr)

#--------------------------------------------------------

ctr_concw["account_no"]=ctr_concw["account_no"].astype(str)
ctr_concw= ctr_concw.query("account_no.str.len() > 8", engine="python")
#--------------------------------------------------------
ctr_concw.groupby(["account_no", "tnx_date"])["account_no"].transform("size")

ctr_concw["transaction_count"] = ctr_concw.groupby(["account_no", "tnx_date"])["account_no"].transform("size")

data2cw = ctr_concw[ctr_concw["transaction_count"] != 1]

data2cw.groupby(["account_no", "tnx_date"])["local_eqv"].transform(sum)

data2cw["sum"]= data2cw.groupby(["account_no", "tnx_date"])["local_eqv"].transform(sum)
data2cw["sum"] =data2cw["sum"].abs()
#--------------------------------------------------------
twomilcw = data2cw[data2cw["sum"] >= 2000000 ]
twomilcw.shape

#----------------- CW CTR WITH GL ----------------------------------------------------------------------------------------------------------------
ctrcw = DTR.copy()
ctr1cw = ctrcw[ctrcw["txn_code"] == 319]
ctr2cw = ctrcw[ctrcw["txn_code"] == 651]
framesctr = [ctr1cw, ctr2cw]
ctr_concw = pd.concat(framesctr)

#--------------------------------------------------------

ctr_concw["account_no"]=ctr_concw["account_no"].astype(str)

#--------------------------------------------------------
ctr_concw.groupby(["account_no", "tnx_date"])["account_no"].transform("size")

ctr_concw["transaction_count"] = ctr_concw.groupby(["account_no", "tnx_date"])["account_no"].transform("size")

data2cw = ctr_concw[ctr_concw["transaction_count"] != 1]

data2cw.groupby(["account_no", "tnx_date"])["local_eqv"].transform(sum)

data2cw["sum"]= data2cw.groupby(["account_no", "tnx_date"])["local_eqv"].transform(sum)
data2cw["sum"] =data2cw["sum"].abs()
#--------------------------------------------------------
twomilcwgl = data2cw[data2cw["sum"] >= 2000000 ]

#----------- vault late FILE UPLOAD -----------------------------------------------------------------------------------------------------------------------------------------------------------

vault = input("Input consolidated vault")
vault = pd.read_excel(vault)


vault_o = vault[['UNIT_ID', 'UNIT_NAME', 'POST_DATE', 'VAULT_OPEN_TIME']]
vault_p = vault[['UNIT_ID', 'UNIT_NAME', 'POST_DATE', 'VAULT_CLOSE_TIME']]


#----------- vault late close-----------------------------------------------------------------------------------------------------------------
vault_p_late = vault_p[vault_p["VAULT_CLOSE_TIME"] > "19:30:00" ]


vault_p_late["dayname"] =vault_p_late["POST_DATE"].dt.strftime('%A')
vault_p_late = vault_p_late[vault_p_late["dayname"] != 'Saturday']



#----------- vault late open-----------------------------------------------------------------------------------------------------------------
vault_o_late = vault_o[vault_o["VAULT_OPEN_TIME"] > "09:03:00" ]
vault_o_late["dayname"] =vault_o_late["POST_DATE"].dt.strftime('%A')
vault_o_late = vault_o_late[vault_o_late["dayname"] != 'Saturday']

#------------- cash in transit -------------------------------------------------------------------------------------------------------------

cit = input("Input consolidated cash in transit")
cit = pd.read_excel(cit)


cit = cit.iloc[13:]
cit .columns = cit .iloc[0]

# Drop the first row
cit  = cit[1:]
cit["Balance"] =cit["Balance"].fillna(0)
cit = cit[cit["Balance"] != 0]

#--------- late utility bill-------------------------------------------------------------------------------------------------------------------------------------

ubill = DTR.copy()

#----------------------------------------------------------------------


values_to_removes = ["fed", "FED", "Fed"]

# Remove the rows where the column value is in the list of values to remove
column_names = "vch_narration"

def contains_fed(value):
    for word in values_to_removes:
        if word in value:
            return True
    return False


#df = df[~df[column_name].apply(contains_fed)]
ubil = ubill[~ubill[column_names].apply(contains_fed)]

#----------------------------------------------------------------------
ubil["vch_narration"] =ubil["vch_narration"].str.lower()


#----------------------------------------------------------------------
column_names = "vch_narration"
values_to_removess = ["bil", "bills", "bill","bils"]
def contains_fed(value):
    for word in values_to_removess:
        if word in value:
            return True
    return False


#df = df[~df[column_name].apply(contains_fed)]
utility_late = ubil[ubil[column_names].apply(contains_fed)]
utility_late = utility_late[utility_late["tr_time"] > "19:00:00" ]
#-------------- KYC IMPORT --------------------------------------------------------------------------------------------------------------

KYC = input("Input  Accounts KYC Review")
KYC = pd.read_excel(KYC)



#KYC_NORMAL
KYC["Account_Status"] = KYC["Account_Status"].str.upper()
KYC["Risk_level"] = KYC["Risk_level"].str.lower()
KYC_NORMAL = KYC[KYC["Account_Status"] == "NORMAL"]

#*-------- ACCOUNT KYC REVIEW BLANK RISK -----------------------------------------------------------------------------------------------------------------
blank = KYC_NORMAL.copy()
bol = blank["Risk_level"].isnull()
blank = blank[bol]


#-------- ACCOUNT KYC REVIEW *high** RISK -----------------------------------------------------------------------------------------------------------------
high = KYC_NORMAL.copy()

high = high[high["Risk_level"] == "high"]

current_date = pd.to_datetime("2023-10-01")


#-------------------------------------------------------
high["datediff"] = high["Revise_Date"]- current_date

#-------------------------------

high_final =high[high["datediff"].dt.days < -365]

#-------- ACCOUNT KYC REVIEW *medium** RISK ----------------------------------------------------------------------------------------------------------------

medium = KYC_NORMAL.copy()
#-----------
medium = medium[medium["Risk_level"] == "medium"]

current_date = pd.to_datetime("2023-07-01")
#-------------------------------------------------------
medium["datediff"] = medium["Revise_Date"]- current_date

#-------------------------------
medium_final =medium[medium["datediff"].dt.days < -730]


#-------- ACCOUNT KYC REVIEW *low** RISK -----------------------------------------------------------------------------------------------------------------

low = KYC_NORMAL.copy()


#-----------

low = low[low["Risk_level"] == "low"]

current_date = pd.to_datetime("2023-07-01")


#-------------------------------------------------------
low["datediff"] = low["Revise_Date"]- current_date

#-------------------------------

low_final =low[low["datediff"].dt.days < -1461]



#---------- donation to source income ------------------------------------------------------------------------------------------------------------
dtn = KYC_NORMAL.copy()
#-----------------------------------------------

dtn["Donation"] = dtn["Donation"].str.upper()
dtn["Source_of_Income"] = dtn["Source_of_Income"].str.upper()

#----------------------------------------------------------
donation_source = dtn[(dtn["Donation"]=="Y") & (dtn["Source_of_Income"] =="Y")]

#----------------------- PO WITHOUT INSTRUMENT --------------------------------------------------------------------------------------------------------------------


pwi = input("Input Remittance Register")
pwi = pd.read_excel(pwi)


#-------------------------------------------------------------
pwi["Status"] =pwi["Status"].str.upper()
#-------------------------------------------------------------
pwi_c = pwi[pwi["Status"] != "C"]
pwi_c_icd = pwi_c[pwi_c['Instrument_Type'] != "ICD"]
#-------------------------------------------------------------
po_without = pwi_c_icd[pwi_c_icd["Cheque_No"] == 0]


#----------------------------------- UNCLAIMED ----------------------------------------------------------------------------------------------------------------------

acc_st = input("Input account status report")
acc_st = pd.read_excel(acc_st)



acc_st["ACCOUNT_STATUS"] = acc_st["ACCOUNT_STATUS"].str.upper()

acc_st = acc_st[acc_st["ACCOUNT_STATUS"] == "DORMANT"]

current_date_a = pd.to_datetime("2023-06-30")


#-------------------------------------------------------
acc_st["datediff"] = acc_st["STATUS_DATE"]- current_date_a

#-------------------------------

unclaimed =acc_st[acc_st["datediff"].dt.days < -3650]


#----------------- PO ABREVIATED ----------------------------------------------------------------------------------------------------------------

po_ab = input("Input Remittance Register_11072023_110712.xls")
po_ab = pd.read_excel(po_ab)



po_ab["Status"] =po_ab["Status"].str.upper()
#-------------------------------------------------------------
po_ab = po_ab[po_ab["Status"] != "C"]
po_ab = po_ab[po_ab['Instrument_Type'] != "ICD"]

#-------------------------------------------------------------

def is_abbreviated(name):
    for i in range(len(name)):
        if not name[i].isalpha() and name[i] != '.':
            return True
    return False
#-------------------------------------------------------------

po_abreviated = po_ab[po_ab["Unit_Name"].apply(is_abbreviated)]


#------------------------ Zakat exemption ------------------------------------------------------------------------------------------------------------------------
ze = input("Zakat Exemption Accounts")
ze = pd.read_excel(ze)




column_name = "Account_Title"

# Get the list of values to remove
values_to_remove = ["Enterprise", "enterprises", "ltd","Limited","limited","CO","corperate","Lancer","Motors","motor","MOTORS"
                   , "Goverment", "goverment","Division","Divisions","divisions","Soft","International","estate","Real","real",
                   "Real estate","contractor","Contractor","Plaza","govt", "COMPANY","trader"]

def contains_fed(value):
    for word in values_to_remove:
        if word in value.lower():
            return True
    return False


#df = df[~df[column_name].apply(contains_fed)]
zakat_exemp = ze[ze[column_name].apply(contains_fed)]


## Create an Excel writer object--------------------------------------------------------------------------------------------
excel_file = "Annexure.xlsx"
writer = pd.ExcelWriter(excel_file, engine='xlsxwriter')

# Dictionary containing DataFrames and corresponding sheet names
dataframes = {
    'Unclaimed': unclaimed,
    'PO without instrument': po_without,
    'Donation to Source income': donation_source,
    'Low risk': low_final, 
    'Medium risk': medium_final, 
    'High risk': high_final, 
    'Blank risk': blank,
    'Utility bill': utility_late,
    'Cash in transit': cit,
    'Vault Open late': vault_o_late,
    'Vault Late close': vault_p_late,
    'CW CTR': twomilcw,
    'CD CTR': twomil,
    'STAN no 641': stan_no_641,
    'Inward clearing': inward_clearing,
    'Withdrawl late': w_late,
    'Deposit late': deposit_late,
    #'PO Abreviated': po_abreviated,
    'Zakat exemption': zakat_exemp

    
    
    # Add more sheets and corresponding DataFrames as needed
}

# Loop through the dictionary and write each DataFrame to a separate sheet
for sheet_name, dataframe in dataframes.items():
    dataframe.to_excel(writer, sheet_name=sheet_name, index=False)

# Save the Excel file
writer.close()



Input consolidated DTRDTR Consolidated.xlsx


<ipython-input-1-ec1d1ab4da62>:17: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  cash_late["tr_time"] = cash_late["tr_time"].astype('datetime64[ns]')
<ipython-input-1-ec1d1ab4da62>:36: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  w_late["tr_time"] = w_late["tr_time"].astype('datetime64[ns]')
<ipython-input-1-ec1d1ab4da62>:51: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  inw["tr_time"] =inw["tr_time"].astype('datetime64[ns]')
<ipython-input-1-ec1d1ab4da62>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_ind

Input consolidated vaultVault Timings - Unit Wise_05102023_101020.xls


<ipython-input-1-ec1d1ab4da62>:177: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vault_o_late["dayname"] =vault_o_late["POST_DATE"].dt.strftime('%A')


Input consolidated cash in transit90113016 Cash in Transit PKR.xls
WARNING *** file size (198642) not 512 + multiple of sector size (512)


<ipython-input-1-ec1d1ab4da62>:217: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ubil["vch_narration"] =ubil["vch_narration"].str.lower()


Input  Accounts KYC ReviewAccounts KYC Review_05102023_091036.xls
Input Remittance RegisterRemittance Register_05102023_101011.xls
Input account status reportAccount Status_05102023_091025.xls
Input Remittance Register_11072023_110712.xlsRemittance Register_05102023_101011.xls
Zakat Exemption AccountsZakat Exemption Accounts_05102023_091059.xls


In [4]:
#extra code 

vault_o_late["dayname"] =vault_o_late["POST_DATE"].dt.strftime('%A')
valts = vault_o_late[vault_o_late["dayname"] != 'Saturday']

vault_o_late["dayname"].unique()

valts = vault_o_late[vault_o_late["dayname"] != 'Saturday']

valts.to_excel("vault_sat.xlsx")

vault_o_late["POST_DATE"].day_name()

vault_o_late["POST_DATE"].dt.day_name()

vault_o_late["POST_DATE"]

vault_o_late[vault_o_late["dayname"] != 'Saturday']

import pandas as pd

vault = pd.read_excel("Vault Timings - Unit Wise_04102023_121042.xls")


vault_o = vault[['UNIT_ID', 'UNIT_NAME', 'POST_DATE', 'VAULT_OPEN_TIME']]

#----------- vault late open-----------------------------------------------------------------------------------------------------------------
vault_o_late = vault_o[vault_o["VAULT_OPEN_TIME"] > "09:03:00" ]
vault_o_late["dayname"] =vault_o_late["POST_DATE"].dt.strftime('%A')
vault_o_late = vault_o_late[vault_o_late["dayname"] != 'Saturday']

vault_o_late 

vault_o_late.to_excel("openlate.xlsx")

#----------- vault late close-----------------------------------------------------------------------------------------------------------------
vault_p_late = vault_p[vault_p["VAULT_CLOSE_TIME"] > "19:30:00" ]

vault_p_late["dayname"] =vault_p_late["POST_DATE"].dt.strftime('%A')
vault_p_late = vault_p_late[vault_p_late["dayname"] != 'Saturday']
vault_p_late

vault_p_late.to_csv("late vault.csv ")